In [1]:
# painting superpixels

import os
import zarr
import napari
import numpy as np
import pandas as pd
import copick
from pathlib import Path
from cellcanvas_spp.segmentation import superpixels
from skimage.measure import regionprops_table
from scipy import stats
from sklearn.ensemble import RandomForestClassifier
from skimage import future
from functools import partial
import threading
import toolz as tz
from psygnal import debounced
from superqt import ensure_main_thread
from qtpy.QtWidgets import (
    QVBoxLayout,
    QHBoxLayout,
    QComboBox,
    QLabel,
    QCheckBox,
    QDoubleSpinBox,
    QGroupBox,
    QWidget,
)
from appdirs import user_data_dir
import logging
import tifffile

# Set up logging
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)

# Set up the data directory
DATA_DIR = Path("/Users/kharrington/git/cellcanvas/superpixels/notebooks/my_synthetic_data_10439_dataportal.json")

# Load the tomogram
def load_tomogram():
    config_file = DATA_DIR
    root = copick.from_file(config_file)
    run_name = "16193"
    example_run = root.get_run(run_name)
    tomogram = example_run.voxel_spacings[0].tomograms[0]
    z = zarr.open(tomogram.zarr())
    img = z["0"]  # Get the highest resolution scale
    return np.asarray(img)

# Load and crop the tomogram
full_tomogram = load_tomogram()
# crop_3D = full_tomogram[50:100, 180:360, 210:430]  # Adjust crop as needed
crop_3D = full_tomogram[:]

# Compute superpixels
# superpixel_seg = superpixels(crop_3D, sigma=4, h_minima=0.0025)
superpixel_seg = tifffile.imread('/Users/kharrington/Downloads/segm_16193.tif')

# Set up Napari viewer
viewer = napari.Viewer()
scale = (1, 1, 1)  # Adjust scale if needed
contrast_limits = (crop_3D.min(), crop_3D.max())

# Add layers
data_layer = viewer.add_image(crop_3D, scale=scale, contrast_limits=contrast_limits, name="Tomogram")
superpixel_layer = viewer.add_labels(superpixel_seg, scale=scale, name="Superpixels", opacity=0.5)

# Set up zarr for prediction and painting layers
zarr_path = os.path.join(user_data_dir("napari_dl_at_mbl_2024", "napari"), "diy_segmentation.zarr")
print(f"zarr path: {zarr_path}")
prediction_data = zarr.open(f"{zarr_path}/prediction", mode='a', shape=crop_3D.shape, dtype='i4', dimension_separator="/")
painting_data = zarr.open(f"{zarr_path}/painting", mode='a', shape=crop_3D.shape, dtype='i4', dimension_separator="/")

prediction_layer = viewer.add_labels(prediction_data, name="Prediction", scale=scale)
painting_layer = viewer.add_labels(painting_data, name="Painting", scale=scale)

# Precompute regionprops features for each superpixel
def compute_superpixel_features(image, superpixels):
    props = regionprops_table(superpixels, intensity_image=image,
                              properties=('label', 
                                          'area', 
                                          'bbox',
                                          'bbox_area',
                                          'centroid',
                                          'equivalent_diameter',
                                          'euler_number',
                                          'extent',
                                          'filled_area',
                                          'major_axis_length',
                                          'max_intensity',
                                          'mean_intensity',
                                          'min_intensity',
                                          'std_intensity',))
    return props

superpixel_features = compute_superpixel_features(crop_3D, superpixel_seg)

def update_model(y, X, model_type):
    logger.debug(f"X shape: {X.shape}")
    logger.debug(f"y shape: {y.shape}")
    logger.debug(f"Unique labels: {np.unique(y)}")
    
    if y.size == 0:
        logger.warning("No labeled data found. Skipping model update.")
        return None
    
    if model_type == "Random Forest":
        clf = RandomForestClassifier(n_estimators=50, n_jobs=-1, max_depth=10, max_samples=0.05)
    
    try:
        clf.fit(X, y)
        logger.info("Model successfully updated")
        return clf
    except Exception as e:
        logger.error(f"Error updating model: {str(e)}")
        return None

def predict(model, superpixel_features):
    features = np.array([[superpixel_features[prop][i] for prop in superpixel_features.keys() if prop != 'label'] 
                         for i in range(len(superpixel_features['label']))])
    prediction = model.predict(features)
    return prediction

# Napari ML Widget
class NapariMLWidget(QWidget):
    def __init__(self, parent=None):
        super(NapariMLWidget, self).__init__(parent)
        self.initUI()

    def initUI(self):
        layout = QVBoxLayout()

        model_label = QLabel("Select Model")
        self.model_dropdown = QComboBox()
        self.model_dropdown.addItems(["Random Forest"])
        model_layout = QHBoxLayout()
        model_layout.addWidget(model_label)
        model_layout.addWidget(self.model_dropdown)
        layout.addLayout(model_layout)

        self.live_fit_checkbox = QCheckBox("Live Model Fitting")
        self.live_fit_checkbox.setChecked(True)
        layout.addWidget(self.live_fit_checkbox)

        self.live_pred_checkbox = QCheckBox("Live Prediction")
        self.live_pred_checkbox.setChecked(True)
        layout.addWidget(self.live_pred_checkbox)

        self.setLayout(layout)

# Add widget to Napari
widget = NapariMLWidget()
viewer.window.add_dock_widget(widget, name="Interactive Segmentation")

# Event listener
model = None

@tz.curry
def on_data_change(event, viewer=None, widget=None):
    painting_layer.refresh()

    thread = threading.Thread(
        target=threaded_on_data_change,
        args=(
            event,
            viewer.dims,
            widget.model_dropdown.currentText(),
            widget.live_fit_checkbox.isChecked(),
            widget.live_pred_checkbox.isChecked(),
        ),
    )
    thread.start()
    thread.join()

    prediction_layer.refresh()

def threaded_on_data_change(
    event,
    dims,
    model_type,
    live_fit,
    live_prediction,
):
    global model, crop_3D, painting_data, superpixel_seg, superpixel_features
    
    # Ensure consistent shapes
    min_shape = [min(s1, s2, s3) for s1, s2, s3 in zip(crop_3D.shape, painting_data.shape, superpixel_seg.shape)]
    logger.debug(f"min_shape: {min_shape}")
    
    active_labels = painting_data[:min_shape[0], :min_shape[1], :min_shape[2]]
    logger.debug("active_labels")
    crop_3D_subset = crop_3D[:min_shape[0], :min_shape[1], :min_shape[2]]
    logger.debug("crop subset")
    superpixel_seg_subset = superpixel_seg[:min_shape[0], :min_shape[1], :min_shape[2]]
    logger.debug("superpixel subset")

    # Recompute superpixel features
    # logger.debug(f"computing superpixel shapes")
    # superpixel_features = compute_superpixel_features(crop_3D_subset, superpixel_seg_subset)
    
    # Create a mask of painted pixels
    painted_mask = active_labels > 0
    
    logger.debug("painted mask")

    if live_fit:
        logger.debug("preparing live fit")
        
        # Create a mask of painted pixels
        painted_mask = active_labels > 0
        
        # Get unique superpixel labels in the painted areas
        painted_superpixels = np.unique(superpixel_seg_subset[painted_mask])
        
        # Prepare features and labels for training
        X = []
        y = []
        
        for label in painted_superpixels:
            mask = superpixel_seg_subset == label
            painted_pixels = active_labels[mask & painted_mask]
            
            if painted_pixels.size > 0:
                feature_vector = [superpixel_features[prop][superpixel_features['label'] == label][0] 
                                for prop in superpixel_features.keys() if prop != 'label']
                X.append(feature_vector)
                y.append(stats.mode(painted_pixels, axis=None)[0])

        X = np.array(X)
        y = np.array(y)
        logger.debug("data prepared for live fit")
        
        logger.debug(f"Number of painted superpixels: {len(X)}")
        logger.debug(f"X shape: {X.shape}, y shape: {y.shape}")
        
        if len(X) > 0:
            model = update_model(y, X, model_type)
        else:
            logger.warning("No painted superpixels found. Skipping model update.")

    if live_prediction and model is not None:
        try:
            logger.debug("Starting prediction")
            
            # Prepare features for all superpixels
            features = np.array([
                [superpixel_features[prop][i] for prop in superpixel_features.keys() if prop != 'label']
                for i in range(len(superpixel_features['label']))
            ])
            
            logger.debug("Starting actual prediction")
            # Predict for all superpixels
            superpixel_predictions = model.predict(features)
            
            logger.debug("Creating mapping")
            # Create a mapping from superpixel label to prediction
            label_to_prediction = dict(zip(superpixel_features['label'], superpixel_predictions))
            
            # Use numpy vectorize to apply the mapping efficiently
            prediction_func = np.vectorize(lambda x: label_to_prediction.get(x, 0))
            prediction = prediction_func(superpixel_seg_subset)
            
            # Ensure prediction has the correct shape and dtype
            prediction = prediction.astype(prediction_layer.data.dtype)
            
            logger.debug("Updating layer")
            # Update the prediction layer data
            prediction_layer.data[:min_shape[0], :min_shape[1], :min_shape[2]] = prediction
            
            logger.debug("Prediction updated successfully")
        except Exception as e:
            logger.error(f"Error during prediction: {str(e)}")
            logger.exception("Detailed traceback:")

# Connect event listeners
for listener in [painting_layer.events.paint]:
    listener.connect(
        debounced(
            ensure_main_thread(
                on_data_change(viewer=viewer, widget=widget)
            ),
            timeout=1000,
        )
    )

napari.run()

DEBUG:gql.dsl:Creating <DSLType <GraphQLObjectType 'datasets'>>)
DEBUG:gql.dsl:Creating <DSLField datasets::id>
DEBUG:gql.dsl:Creating <DSLField datasets::cell_component_id>
DEBUG:gql.dsl:Creating <DSLField datasets::cell_component_name>
DEBUG:gql.dsl:Creating <DSLField datasets::cell_name>
DEBUG:gql.dsl:Creating <DSLField datasets::cell_strain_id>
DEBUG:gql.dsl:Creating <DSLField datasets::cell_strain_name>
DEBUG:gql.dsl:Creating <DSLField datasets::cell_type_id>
DEBUG:gql.dsl:Creating <DSLField datasets::dataset_citations>
DEBUG:gql.dsl:Creating <DSLField datasets::dataset_publications>
DEBUG:gql.dsl:Creating <DSLField datasets::deposition_date>
DEBUG:gql.dsl:Creating <DSLField datasets::description>
DEBUG:gql.dsl:Creating <DSLField datasets::grid_preparation>
DEBUG:gql.dsl:Creating <DSLField datasets::https_prefix>
DEBUG:gql.dsl:Creating <DSLField datasets::key_photo_thumbnail_url>
DEBUG:gql.dsl:Creating <DSLField datasets::key_photo_url>
DEBUG:gql.dsl:Creating <DSLField datasets::l

zarr path: /Users/kharrington/Library/Application Support/napari_dl_at_mbl_2024/diy_segmentation.zarr


DEBUG:napari.layers.base.base:Layer._slice_dims: Painting, dims=ndim=3 ndisplay=2 order=(0, 1, 2) axis_labels=('0', '1', '2') range=(RangeTuple(start=0.0, stop=199.0, step=1.0), RangeTuple(start=0.0, stop=629.0, step=1.0), RangeTuple(start=0.0, stop=629.0, step=1.0)) margin_left=(0.0, 0.0, 0.0) margin_right=(0.0, 0.0, 0.0) point=(99.0, 314.0, 314.0) last_used=0, force=False
DEBUG:napari.layers.base.base:Layer._refresh_sync: Painting


DEBUG:in_n_out:Executing @injected activate_labels_paint_mode(layer: napari.layers.labels.labels.Labels) with args: (<Labels layer 'Painting' at 0x1c9a3cdf0>,), kwargs: {}
DEBUG:in_n_out:  Calling activate_labels_paint_mode with {'layer': <Labels layer 'Painting' at 0x1c9a3cdf0>} (injected set())
DEBUG:napari.layers.base.base:Layer.refresh: Painting
DEBUG:napari.layers.base.base:Layer._refresh_sync: Painting
DEBUG:__main__:min_shape: [200, 630, 630]
DEBUG:__main__:active_labels
DEBUG:__main__:crop subset
DEBUG:__main__:superpixel subset
DEBUG:__main__:painted mask
DEBUG:__main__:preparing live fit
DEBUG:__main__:data prepared for live fit
DEBUG:__main__:Number of painted superpixels: 7
DEBUG:__main__:X shape: (7, 20), y shape: (7,)
DEBUG:__main__:X shape: (7, 20)
DEBUG:__main__:y shape: (7,)
DEBUG:__main__:Unique labels: [1]
INFO:__main__:Model successfully updated
DEBUG:__main__:Starting prediction
DEBUG:__main__:Prediction updated successfully
DEBUG:napari.layers.base.base:Layer.refr

In [3]:
superpixel_features.keys()


dict_keys(['label', 'area', 'bbox-0', 'bbox-1', 'bbox-2', 'bbox-3', 'bbox-4', 'bbox-5', 'bbox_area', 'centroid-0', 'centroid-1', 'centroid-2', 'equivalent_diameter', 'euler_number', 'extent', 'filled_area', 'major_axis_length', 'max_intensity', 'mean_intensity', 'min_intensity', 'std_intensity'])

DEBUG:in_n_out:Executing @injected reset_scroll_progress(viewer: napari.viewer.Viewer) with args: (Viewer(camera=Camera(center=(0.0, 44.531128057650676, 390.16229341195395), zoom=0.8985877242862453, angles=(0.0, 0.0, 90.0), perspective=0.0, mouse_pan=False, mouse_zoom=True), cursor=Cursor(position=(39.0, 366.1469385372584, 250.4986948619642), scaled=True, style=<CursorStyle.CIRCLE: 'circle'>, size=2.0), dims=Dims(ndim=3, ndisplay=2, order=(0, 1, 2), axis_labels=('0', '1', '2'), range=(RangeTuple(start=0.0, stop=199.0, step=1.0), RangeTuple(start=0.0, stop=629.0, step=1.0), RangeTuple(start=0.0, stop=629.0, step=1.0)), margin_left=(0.0, 0.0, 0.0), margin_right=(0.0, 0.0, 0.0), point=(39.0, 314.0, 314.0), last_used=0), grid=GridCanvas(stride=1, shape=(-1, -1), enabled=False), layers=[<Image layer 'Tomogram' at 0x1c82da6e0>, <Labels layer 'Superpixels' at 0x1c8368c10>, <Labels layer 'Prediction' at 0x1c9a3cc40>, <Labels layer 'Painting' at 0x1c9a3cdf0>], help='use <6> for pan/zoom, use <7